<a href="https://colab.research.google.com/github/vineelbhatti/NbaTeamFit/blob/main/NbaTeamFit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder

# Load the college basketball player stats dataset
nba_stats = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/2022-23_NBA_Player_Stats.csv', encoding = 'latin', sep=';')
height_weight = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/HeightAndWeight.csv')
height_weight.rename(columns={'Name':'Player'}, inplace=True)
nba_stats = pd.merge(nba_stats, height_weight, on='Player')
nba_stats = nba_stats.drop(columns=['Age_y', 'Height', 'Points', 'Rebounds', 'Assists', 'Team', 'Position'])
nba_stats.head(10)




,Rk,Player,Pos,Age_x,Tm,G,GS,MP,FG,FGA,...,AST,STL,BLK,TOV,PF,PTS,Height_i,Weight,College,Salary
0,1,Precious Achiuwa,C,23,TOR,33,9,23.0,4.0,8.2,...,1.1,0.7,0.7,1.2,2.2,10.4,6.80,225,Memphis,2711280.0
1,2,Steven Adams,C,29,MEM,42,42,27.0,3.7,6.3,...,2.3,0.9,1.1,1.9,2.3,8.6,6.11,265,Pittsburgh,17073171.0
2,3,Bam Adebayo,C,25,MIA,52,52,35.3,8.6,15.7,...,3.3,1.2,0.8,2.6,2.8,21.6,6.90,255,Kentucky,28103500.0
3,5,Santi Aldama,PF,22,MEM,52,18,22.0,3.4,7.0,...,1.2,0.7,0.7,0.7,1.9,9.5,6.11,215,Loyola (MD),1994520.0
4,6,Nickeil Alexander-Walker,SG,24,TOT,37,3,14.3,2.2,4.6,...,2.1,0.6,0.4,1.2,1.6,6.2,6.60,205,Virginia Tech,3261480.0
5,6,Nickeil Alexander-Walker,SG,24,UTA,36,3,14.7,2.3,4.7,...,2.1,0.7,0.4,1.3,1.6,6.3,6.60,205,Virginia Tech,3261480.0
6,6,Nickeil Alexander-Walker,SG,24,MIN,1,0,2.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.60,205,Virginia Tech,3261480.0
7,7,Grayson Allen,SG,27,MIL,52,50,27.5,3.4,7.6,...,2.4,0.8,0.2,1.1,1.6,10.4,6.40,198,Duke,4054695.0
8,8,Jarrett Allen,C,24,CLE,52,52,33.1,6.2,9.4,...,1.7,0.8,1.2,1.6,2.2,14.7,6.11,243,Texas,20000000.0
9,10,Kyle Anderson,PF,29,MIN,46,31,26.0,3.0,6.0,...,4.0,1.2,0.8,1.4,2.2,8.3,6.90,230,UCLA,9937150.0


In [ ]:
# Load a list of NBA team names
nba_teams = ['ATL', 'BOS', 'BKN', 'CHA', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOP', 'NYK', 'OKC', 'ORL', 'PHI', 'PHX', 'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS']

nba_stats = nba_stats.rename({'FG%': 'FG'})

# Define the feature columns
feature_cols = ['G', 'PTS', 'ORB', 'DRB', 'AST', 'FG%', 'BLK', 'STL', 'Height_i', 'Weight']

# Split the data into training and testing sets
X_train = nba_stats[feature_cols]
y_train = nba_stats['Tm']

# Create the KNN model
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Prompt the user to enter their college stats
G = int(input("Enter the number of games played: "))
PTS = float(input("Enter the average points per game: "))
ORB = float(input("Enter the average offensive rebounds per game: "))
DRB = float(input("Enter the average defensive rebounds per game: "))
AST = float(input("Enter the average assists per game: "))
FG = float(input("Enter the field goal percentage: "))
BLK = float(input("Enter the avaerage blocks per game: "))
STL = float(input("Enter the average steals per game: "))
Height_i = float(input("Enter the height in decimal format: "))
Weight = float(input("Enter the weight in pounds: "))

# Use the KNN model to predict the NBA team fit
player_stats = [[G, PTS, ORB, DRB, AST, FG, BLK, STL, Height_i, Weight]]
distances, indices = knn.kneighbors(player_stats)
team_name = knn.predict(player_stats)[0]
team_index = nba_teams.index(team_name)
nba_team = nba_teams[team_index]

# Print the predicted NBA team fit
print("Based on your college stats, the best NBA team fit for you is:", nba_team)

print("Here are some similar players from the college_stats dataset:")
for index in indices[0]:
    print(nba_stats.loc[index, 'Player'])



Enter the number of games played: 72
Enter the average points per game: 22.5
Enter the average offensive rebounds per game: 0.2
Enter the average defensive rebounds per game: 0.3
Enter the average assists per game: 8.7
Enter the field goal percentage: 47.6
Enter the avaerage blocks per game: 0.2
Enter the average steals per game: 4.5
Enter the height in decimal format: 6.2
Enter the weight in pounds: 190
Based on your college stats, the best NBA team fit for you is: GSW
Here are some similar players from the college_stats dataset:
Jordan Poole
Jordan Clarkson
D'Angelo Russell
Jalen Brunson
D'Angelo Russell


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
